# Maximisation couple moyen

Ce notebook propose d'optimiser le rotor d'une machine synchro-réluctante sans aimant à une paire de pôle en vue de maximiser le couple moyen.
Dans ce cas particulier, le couple moyen est directement relié à la différence de perméance magnétique du rotor dans l'axe direct d et l'axe en quadrature q.

Le problème étudié est donc la maximisation de cette différence de perméance via la différence de compliance magnétique. Il est possible de démontrer que ce problème est mal posé.

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry
#from netgen.geom2d import unit_square
import copy
from numpy.random import randint
import numpy as np
import matplotlib.pyplot as plt

importing NGSolve-6.2.2105-9-g5a835126f


## 1) Définition du maillage

La première étape consiste à définir une géométrie (disque), ainsi que la finesse de maillage associée.

In [2]:
list(np.array([0,0.5,*(0.5+np.arange(1,10-1)),10-1])/(10-1))

[0.0,
 0.05555555555555555,
 0.16666666666666666,
 0.2777777777777778,
 0.3888888888888889,
 0.5,
 0.6111111111111112,
 0.7222222222222222,
 0.8333333333333334,
 0.9444444444444444,
 1.0]

In [3]:
def generate_fitted_mesh_square(N):
    h=min(1/(3*N),1/50)
    geo = SplineGeometry()
    X = list(np.array([0,0.5,*(0.5+np.arange(1,N-1)),N-1])/(N-1))
    #X = [0,*np.linspace(1/(2*(N-2)),1-1/(2*(N-2)),N-1),1]
    Y = [0,1]

    p = [ geo.AppendPoint(x,y) for x in X for y in Y ]

    geo.Append (["line", p[0], p[1]], leftdomain=0, rightdomain=1,bc="left")
    geo.Append (["line", p[0], p[2]], leftdomain=1, rightdomain=0,bc="bottom")
    geo.Append (["line", p[1], p[3]], leftdomain=0, rightdomain=1,bc="up")
    geo.SetMaterial(1,'iron')
    mat = ['iron','air']
    for i in range(2,2*N-1,2):
        geo.Append (["line", p[i], p[i+1]], leftdomain=i//2, rightdomain=i//2+1)
        geo.Append (["line", p[i], p[i+2]], leftdomain=i//2+1, rightdomain=0,bc="bottom")
        geo.Append (["line", p[i+1], p[i+3]], leftdomain=0, rightdomain=i//2+1,bc="up")
        geo.SetMaterial(i//2+1,mat[(i//2)%2])
    
    geo.Append (["line", p[2*N], p[2*N+1]], leftdomain=N, rightdomain=0,bc="right")

    ngmesh = geo.GenerateMesh(maxh=h)
    mesh = Mesh(ngmesh)
    return(mesh)

mesh = generate_fitted_mesh_square(20)
Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2105-9-g5a835126f', 'mesh_dim': 2, 'order2d': 1, 'order3d': 1, 'dr…

BaseWebGuiScene

## 2) Définition des problèmes éléments finis

L'objectif est ici de maximiser la différence de compliance magnétique pour deux chargement distincts. On va résoudre le problème magnétostatique dual ; pour cela, remarquons qu'en l'absence de termes sources:

$$ \overrightarrow{\text{rot}} ~ \overrightarrow{h} = 0 $$

On pose donc $\phi$ un potentiel scalaire tel que $\overrightarrow{h} = \overrightarrow{\text{grad}} ~ \phi$, qui vérifie donc dans $\Omega$:
$$ \text{div}( \mu ~\overrightarrow{\text{grad}} ~ \phi ) = 0 $$

$\phi$ est un champ scalaire qu'on va supposer appartenant à $H^1$. Les conditions aux bords sont de Neumann. La solution est unique à une constante près. Les deux problèmes à résoudre s'écrivent donc :

$$\left \{
\begin{array}{l}
     \text{div}( \mu ~\overrightarrow{\text{grad}} ~ \phi ) = 0 \\
     \frac{\partial \phi}{\partial n} = \Phi ~ \text{sur} ~ \Gamma_1 \bigcup \Gamma_2 \\
     \frac{\partial \phi}{\partial n} = - \Phi ~ \text{sur} ~ \Gamma_3 \bigcup \Gamma_4 
\end{array}
\right. $$

$$\left \{
\begin{array}{l}
     \text{div}( \mu ~\overrightarrow{\text{grad}} ~ \phi ) = 0 \\
     \frac{\partial \phi}{\partial n} = \Phi ~ \text{sur} ~ \Gamma_2 \bigcup \Gamma_3 \\
     \frac{\partial \phi}{\partial n} = - \Phi ~ \text{sur} ~ \Gamma_1 \bigcup \Gamma_4 
\end{array}
\right. $$


Sous forme faible, ce problème s'écrit donc :

$$\forall \psi \in H^1, \underbrace{\int_{\Omega} ~\overrightarrow{\text{grad}} ~ \psi \cdot  \mu ~\overrightarrow{\text{grad}} ~ \phi}_{\text{forme bilinéaire} } = \underbrace{\int_{\Gamma_+} \psi \Phi - \int_{\Gamma_-} \psi \Phi}_{\text{forme linéaire}} $$

Par ailleurs, on suppose que $\mu$ est constante par morceau.

In [4]:
PHI = 1;

def solvePb(mu):
    fespace_H1 = H1(mesh, order=1)
    fespace_H1.FreeDofs()[0] = False
    phi = fespace_H1.TrialFunction()
    psi = fespace_H1.TestFunction()
    K = BilinearForm(fespace_H1, symmetric=True)
    K +=  grad(psi)*mu*grad(phi)*dx

    n= specialcf.normal(mesh.dim);

    l1 = LinearForm(fespace_H1)
    l1 += -psi*PHI*ds(definedon=mesh.Boundaries("bottom"))
    l1 += psi*PHI*ds(definedon=mesh.Boundaries("up"))

    l2 = LinearForm(fespace_H1)
    l2 += -psi*PHI*ds(definedon=mesh.Boundaries("left"))
    l2 += psi*PHI*ds(definedon=mesh.Boundaries("right"))
    
    K.Assemble()
    l1.Assemble()
    l2.Assemble()
    
    phi1 = GridFunction(fespace_H1)  # solution
    phi1.vec.data =     K.mat.Inverse(inverse="sparsecholesky") * l1.vec
    phi2 = GridFunction(fespace_H1)  # solution
    phi2.vec.data =     K.mat.Inverse(inverse="sparsecholesky") * l2.vec
    
    return(phi1, phi2)
    
fespace_mu = L2(mesh, order=0)    
mu = GridFunction(fespace_mu)
s = (randint(3, size=len(mu.vec[:].FV()))+1)-2
mu.vec[:] = 4e-7*3.14*1.5 #(2+s[:])

In [5]:
phi1, phi2 = solvePb(mu)

In [6]:
2//2

1

In [7]:
Draw (phi2, mesh);

WebGuiWidget(value={'ngsolve_version': '6.2.2105-9-g5a835126f', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'dr…

## 3) Optimisation

Le problème est auto-adjoint. Ainsi, l'adjoint s'écrit $\psi = -\phi$ et l'identification $\mathcal{L}^2$ du gradient par rapport à $\mu$ s'écrit:
$$\partial_\mu \mathcal{L} = -|\nabla \phi|^2 $$

In [8]:
def gradient(phi1,phi2):
    return(grad(phi1)*grad(phi1) + grad(phi2)*grad(phi2))

def compliance(phi,mu):
    return Integrate(grad(phi)*mu* grad(phi)* dx,mesh)

In [10]:
J1=[]
J2=[]
mu0 = 4e-7*3.14
mu_max = 3* mu0
for i in range(5,40):
    mesh = generate_fitted_mesh_square(i)
    mur = mesh.MaterialCF({ "iron" : 1000 }, default=1)
    mu = mu0*mur
    phi1, phi2 = solvePb(mu)
    
    j1 = compliance(phi1,mu);
    j2 = compliance(phi2,mu);
    print([j1,j2])
    J1.append(copy.copy(j1))
    J2.append(copy.copy(j2))
    
plt.plot(list(range(4,40,2)),J1)
plt.plot(list(range(4,40,2)),J2)

[54252.36104143778, 398487.26114606723]
[43227.97849495867, 398487.26114480203]
[35767.91506928748, 398487.26114858006]
[30541.825358414975, 398487.2611382977]
[26454.30231406279, 398487.26114624704]
[23472.922703819022, 398487.26114449435]
[20773.7133875027, 398487.2611461824]
[18628.16421249286, 398487.26114682714]
[16925.6704598228, 398487.26114662783]
[15191.888869812328, 398487.2611421966]
[14078.77194613635, 398487.26114460506]
[12648.316655820536, 398487.2611451329]
[11904.139459286795, 398487.26114525]
[11231.146488260401, 398487.26114386326]
[10747.00649732805, 398487.2611397588]
[10074.824502949783, 398487.2611462677]
[9842.008563651683, 398487.2611446917]
[9326.399172207839, 398487.26114677044]
[9121.449369783692, 398487.2611409768]


KeyboardInterrupt: 

In [ ]:
Draw(phi1,mesh)

In [ ]:
Draw(grad(phi1)*mu*grad(phi1),mesh)

In [ ]:
Draw(10*mu+g,mesh)

In [ ]:
Draw(InnerProduct(-grad(phi1),grad(phi1))+InnerProduct(grad(phi2),grad(phi2)),mesh)

In [ ]:
help(Draw)

In [ ]:
phi1, phi2 = solvePb(mu)
g = gradient(phi1,phi2)
Draw(mu,mesh)
a = mu+g

In [ ]:
Draw(gradient(phi1,phi2),mesh)

In [ ]:
Integrate((grad(phi1)*mu* grad(phi1) - grad(phi2)*mu* grad(phi2))*dx)

In [ ]:
(grad(phi1)*mu* grad(phi1) - grad(phi2)*mu* grad(phi2))*dx

In [ ]:
grad(phi1)*mu* grad(phi1) - grad(phi2),mu* grad(phi2)

In [ ]:
g.vec[:]= g.vec[:].FV().NumPy()/max(g.vec[:])

In [ ]:
Integrate(grad(phi1)*mu* grad(phi1) - grad(phi2)*mu* grad(phi2), mesh)

In [ ]:
J=[]
step = 0.1
g = GridFunction(fespace_mu)

In [ ]:
help(Draw)

In [ ]:
a.datavec.FV()[:] = a.vec.FV()[:] /max(a.vec.FV()[:] )